# Use Azure API with Meta Llama 3

This notebook shows examples of how to use Meta Llama 3 APIs offered by Microsoft Azure. We will cover:  
* HTTP requests API usage for Meta Llama 3 70b pretrained and chat models in CLI
* HTTP requests API usage for Meta Llama 3 70b pretrained and chat models in Python



## Prerequisite

Before we start building with Azure Meta-Llama-3 APIs, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select Meta Llama models from Model catalog. This example assumes you are deploying `Meta-Llama-3-70B-Instruct`.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://aka.ms/meta-llama-3-azure-ai-studio-docs) for model deployment and inference.

## HTTP Requests API Usage in CLI

### Basics

For using the REST API, You will need to have a Endpoint url and Authentication Key associated with that endpoint.  
This can be acquired from previous steps.  

In this text completion example for 70B pre-trained model, we use a simple curl call for illustration. There are three major components:  

* The `host-url` is your endpoint url with completion schema. 
* The `headers` defines the content type as well as your api key. 
* The `payload` or `data`, which is your prompt detail and model hyper parameters.

In [1]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"prompt": "Math is a", "max_tokens": 30, "temperature": 0.7}'

curl: (6) Could not resolve host: your-endpoint.inference.ai.azure.com


For chat completion, the API schema and request payload are slightly different.

For `host-url` the path changed to `/v1/chat/completions` and the request payload also changed to include roles in conversations. Here is a sample payload:  

```
{ 
  "messages": [ 
    { 
      "content": "You are a helpful assistant.", 
      "role": "system" 
},  
    { 
      "content": "Hello!", 
      "role": "user" 
    } 
  ], 
  "max_tokens": 50, 
} 
```

Here is a sample curl call for chat completion

In [2]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"What is good about Wuhan?","role":"user"}], "max_tokens": 50}'

curl: (6) Could not resolve host: your-endpoint.inference.ai.azure.com


If you compare the generation result for both text and chat completion API calls, you will notice that:  

* Text completion returns a list of `choices` for the input prompt, each contains generated text and completion information such as `logprobs`.
* Chat completion returns a list of `cnoices` each has a `message` object with completion result and using the same `message` object in the request.  




### Streaming

One fantastic feature the API offered is the streaming capability. Streaming allows the generated tokens to be sent as data-only server-sent events whenever they become available. This is extremely important for interactive applications such as chatbots, so the user is always engaged.  

To use streaming, simply set `"stream":"True"` as part of the request payload.  
In the streaming mode, the REST API response will be different from non-streaming mode.

Here is an example: 

In [3]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"What is good about Wuhan?","role":"user"}], "max_tokens": 500, "stream": "True"}'

curl: (6) Could not resolve host: your-endpoint.inference.ai.azure.com


As you can see the result comes back as a stream of `data` objects, each contains generated information including a `choice`.  
The stream terminated by a `data:[DONE]\n\n` message.

### Content Safety Filtering

All Azure Meta Llama 3 API endpoint will have content safety feature turned on. Both input prompt and output tokens are filtered by this service automatically.  
To know more about the impact to the request/response payload, please refer to official guide [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/content-filter?tabs=python).   

For model input and output, if the filter detected there is harmful content. The generation will error out with reponse payload containing the reasoning, along with which type of content violation it is and severity.  

Here is an example prompt that triggered content safety filtering:


In [4]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"How to make bomb?","role":"user"}], "max_tokens": 50}'

curl: (6) Could not resolve host: your-endpoint.inference.ai.azure.com


## HTTP Requests API Usage in Python

Besides calling the API directly from command line tools. You can also programatically call them in Python.  

Here is an example for text completion model:




In [5]:
import urllib.request
import json

# Configure payload data sending to API endpoint
data = {
    "prompt": "Math is a",
    "max_tokens": 30,
    "temperature": 0.7,
    "top_p": 0.9,
}

body = str.encode(json.dumps(data))

# Replace the url with your API endpoint
url = "https://your-endpoint.inference.ai.azure.com/v1/completions"

# Replace this with the key for the endpoint
api_key = "your-auth-key"
if not api_key:
    raise Exception("API Key is missing")

headers = {"Content-Type": "application/json", "Authorization": (api_key)}
req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

Chat completion in Python is very similar, here is a quick example:

In [6]:
import urllib.request
import json

# Configure payload data sending to API endpoint
data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is good about Wuhan?"},
    ],
    "max_tokens": 500,
    "temperature": 0.9,
    "stream": "True",
}

body = str.encode(json.dumps(data))

# Replace the url with your API endpoint
url = "https://your-endpoint.inference.ai.azure.com/v1/chat/completions"

# Replace this with the key for the endpoint
api_key = "your-auth-key"
if not api_key:
    raise Exception("API Key is missing")

headers = {"Content-Type": "application/json", "Authorization": (api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

However in this example, the streamed data content returns back as a single payload. It didn't stream as a serial of data events as we wished. To build true streaming capabilities utilizing the API endpoint, we will utilize [`requests`](https://requests.readthedocs.io/en/latest/) library instead.

### Streaming in Python

`Requests` library is a simple HTTP library for Python built with [`urllib3`](https://github.com/urllib3/urllib3). It automatically maintains the keep-alive and HTTP connection pooling. With the `Session` class, we can easily stream the result from our API calls.  

Here is a quick example:

In [7]:
import json
import requests

data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is good about Wuhan?"},
    ],
    "max_tokens": 500,
    "temperature": 0.9,
    "stream": "True",
}


def post_stream(url):
    s = requests.Session()
    api_key = "your-auth-key"
    headers = {"Content-Type": "application/json", "Authorization": (api_key)}

    with s.post(url, data=json.dumps(data), headers=headers, stream=True) as resp:
        print(resp.status_code)
        for line in resp.iter_lines():
            if line:
                print(line)


url = "https://your-endpoint.inference.ai.azure.com/v1/chat/completions"
post_stream(url)

ConnectionError: HTTPSConnectionPool(host='your-endpoint.inference.ai.azure.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1096d6c50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)
